# Limitations

## Description
This model and the machine learning tools used are **flawed** and intended for **research purposes only**.  
Predictions from these models may have an error of approximately **3–4%**.  
Please be patient while I work on finding the right algorithm and fixing it. :))

# Set up


In [1]:
import sys 
import os
notebook_directory = os.getcwd()
module_path = os.path.abspath(os.path.join(notebook_directory, "../models/src/"))
sys.path.append(module_path)
from model_training import model_tuning_for_linear_regression, model_tuning_for_random_forest_regressor, model_training_based_on_linear_regression, model_training_based_on_random_forest_regressor

# Hyperparameter Tuning for Your Model
We will perform **hyperparameter tuning** for our dataset using the following models:

- **Linear Regression**
- **Random Forest Regressor**

In [2]:
print(model_tuning_for_linear_regression())
print(model_tuning_for_random_forest_regressor())

(LinearRegression(copy_X=False, n_jobs=15), {'positive': False, 'n_jobs': 15, 'fit_intercept': True, 'copy_X': False})
(RandomForestRegressor(max_depth=10, min_samples_split=5, n_estimators=500), {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 10, 'bootstrap': True})


# Linear Regression Model for Current Day

## Description
This model predicts the **low and high temperatures** for the current date (today).

In [3]:
model_training_based_on_linear_regression()

'low temperature: 19.156725114019807, high temperature: 23.606985883130207'

# Random Forest Regressor Model for Current Day

## Description
Similar to the Linear Regression model, this model predicts the **low and high temperatures** for the current day.

In [4]:
model_training_based_on_random_forest_regressor()

'low temperature: 16.400467587448656, high temperature: 20.571017023655454'